In [135]:
import pandas as pd
import geopandas as gpd
import pyproj
import math
import numpy as np
import scipy
from sklearn.cluster import DBSCAN
from sklearn.linear_model import LinearRegression
from geopy.distance import great_circle, lonlat, distance, geodesic
from shapely import Point, set_srid, get_srid, box
from datetime import datetime
import folium

In [2]:
df = pd.read_csv("scanned_aps_sanitized.csv", usecols=["created_at", "ssid", "bssid", "level", "latitude", "longitude"])
df["created_at"] = pd.to_datetime(df["created_at"], format="ISO8601")

## Michael's place -- box (47.552425, -122.266685) (47.550714, -122.262940)
df = df.loc[(df["latitude"] < 47.552425) & (df["latitude"] >= 47.550714) & (df["longitude"] >= -122.266685 ) & (df["longitude"] < -122.262940)]


df["lonlat"] = df.apply(lambda x: Point(x.longitude, x.latitude), axis=1)

gdf = gpd.GeoDataFrame(df, geometry="lonlat")
gdf.set_crs(epsg=4326, inplace=True)

gdf = gdf.loc[gdf["ssid"] == "Ernie"].copy()

In [3]:
bssids = gdf["bssid"].unique()

bssid = bssids[0]

bdf = gdf.loc[gdf["bssid"] == bssid]

bdf

,created_at,latitude,longitude,ssid,bssid,level,lonlat
3492,2024-03-31 20:02:15.697756,47.55157,-122.26533,Ernie,d0:21:f9:63:55:b0,-49,POINT (-122.26533 47.55157)
5217,2024-03-31 21:28:19.209850,47.55161,-122.26529,Ernie,d0:21:f9:63:55:b0,-58,POINT (-122.26529 47.55161)
5433,2024-03-31 23:59:03.083500,47.55158,-122.26534,Ernie,d0:21:f9:63:55:b0,-67,POINT (-122.26534 47.55158)
5602,2024-03-31 21:50:30.261954,47.55157,-122.26530,Ernie,d0:21:f9:63:55:b0,-54,POINT (-122.2653 47.55157)
6896,2024-03-31 23:11:31.736455,47.55158,-122.26533,Ernie,d0:21:f9:63:55:b0,-48,POINT (-122.26533 47.55158)
...,...,...,...,...,...,...,...
288098,2024-05-16 05:42:28.201137,47.55163,-122.26517,Ernie,d0:21:f9:63:55:b0,-44,POINT (-122.26517 47.55163)
293112,2024-05-16 06:19:51.219706,47.55164,-122.26517,Ernie,d0:21:f9:63:55:b0,-72,POINT (-122.26517 47.55164)
293774,2024-05-16 06:28:17.545434,47.55163,-122.26517,Ernie,d0:21:f9:63:55:b0,-72,POINT (-122.26517 47.55163)
303174,2024-05-16 08:17:55.307316,47.55164,-122.26516,Ernie,d0:21:f9:63:55:b0,-65,POINT (-122.26516 47.55164)


# Preprocessing Phase

Embed the GPS coordinate into a 2D system. 
Place a unit meter grid on the map, and average all RSS and coordinates that fall into the same grid square.



In [83]:
bdf = bdf.to_crs(epsg=3857) # I believe I can use PostgreSQL to compute this in the app.
bdf["x"] = bdf.lonlat.x
bdf["y"] = bdf.lonlat.y

def make_grid(df, size):
    pass

GRID_SIZE = 5.0 # meters
xmin, ymin, xmax, ymax = bdf.total_bounds 

cols = math.ceil((xmax - xmin) / GRID_SIZE)
rows = math.ceil((ymax - ymin) / GRID_SIZE)

grids = []
grids_data = []

for row in range(rows):
    for col in range(cols):
        g_xmin = xmin + (col * GRID_SIZE)
        g_xmax = xmin + ((col + 1) * GRID_SIZE)
        g_ymin = ymin + (row * GRID_SIZE)
        g_ymax = ymin + ((row + 1) * GRID_SIZE)

        grid_data = bdf.loc[
            (bdf["x"] >= g_xmin) & (bdf["x"] < g_xmax) & 
            (bdf["y"] >= g_ymin) & (bdf["y"] < g_ymax)
        ]
        grids.append({
            "geom": box(g_xmin, g_ymin, g_xmax, g_ymax), 
            "name": f"({row}, {col})"
        })
        if grid_data.index.empty:
            continue
            
        mean_level = grid_data["level"].mean()
        mean_coordinate = grid_data["lonlat"].centroid.iloc[0]
        grids_data.append({
            "level": mean_level, 
            "coordinate": mean_coordinate, 
            "x": mean_coordinate.x,
            "y": mean_coordinate.y,
            "ssid": grid_data.iloc[0]["ssid"], 
            "bssid": grid_data.iloc[0]["bssid"]
        })



grids_df = gpd.GeoDataFrame(grids, geometry="geom", crs=bdf.crs)
grids_data = gpd.GeoDataFrame(grids_data, geometry="coordinate", crs=bdf.crs)



In [84]:
grids_data

,level,coordinate,x,y,ssid,bssid
0,-58.0,POINT (-13610436.354 6032489.559),-1.361044e+07,6.032490e+06,Ernie,d0:21:f9:63:55:b0
1,-67.0,POINT (-13610326.147 6032492.858),-1.361033e+07,6.032493e+06,Ernie,d0:21:f9:63:55:b0
2,-72.0,POINT (-13610284.959 6032489.559),-1.361028e+07,6.032490e+06,Ernie,d0:21:f9:63:55:b0
3,-64.0,POINT (-13610586.635 6032499.455),-1.361059e+07,6.032499e+06,Ernie,d0:21:f9:63:55:b0
4,-54.0,POINT (-13610538.768 6032499.455),-1.361054e+07,6.032499e+06,Ernie,d0:21:f9:63:55:b0
...,...,...,...,...,...,...
56,-77.0,POINT (-13610507.598 6032593.468),-1.361051e+07,6.032593e+06,Ernie,d0:21:f9:63:55:b0
57,-68.6,POINT (-13610496.466 6032590.169),-1.361050e+07,6.032590e+06,Ernie,d0:21:f9:63:55:b0
58,-50.0,POINT (-13610479.768 6032591.819),-1.361048e+07,6.032592e+06,Ernie,d0:21:f9:63:55:b0
59,-50.0,POINT (-13610490.9 6032595.117),-1.361049e+07,6.032595e+06,Ernie,d0:21:f9:63:55:b0


In [85]:
m = folium.Map(location=[47.5515, -122.265], zoom_start=21)

popup = folium.GeoJsonPopup(fields=["name"])
folium.GeoJson(
    grids_df.to_crs(epsg=4326).to_json(),
    name="name",
    popup=popup,
).add_to(m)


for _, row in grids_data.to_crs(epsg=4326).iterrows():
    folium.Marker(
        location=[row["coordinate"].y, row["coordinate"].x],
        tooltip=f"Localized: {row["ssid"]} - {row["bssid"]}",
        popup=f'level: {row["level"]}',
        icon=folium.Icon(color="red")
    ).add_to(m)


m

## Drawing the arrows

Calculate the direction in which the RSS increases the most. 

1. Pick each grid with measurement, and define it as the center of a plane we'll fit.
2. Define a "window size", it is half the length of the side of the square of this plane.
3. Gather all measurements that fall within this window, set them as near-by measurements in the x-y-RSS space and fit a plane from it.
4. Take the gradient at the center of the plane

In [129]:
WINDOW_SIZE = 4 * GRID_SIZE; # results in a 10x10 meters square
MIN_POINTS = 5 # Minimum number of dots required to fit a plane


def fit_plane_mmse(center_point, points):
    X = points[:, :2]
    # Add intercept
    X = np.hstack((X, np.ones((X.shape[0], 1))))
    y = points[:, 2]

    # Compute the Minimum Mean-Square Error equation
    theta = np.linalg.inv((X.T.dot(X))).dot(X.T).dot(y)

    # The fitted plane must pass through the center point
    # so we must adjust our intercept component (c parameter), to make it match exactly the z value at the center point
    z_est = (theta[0]*center_point[0] + theta[1]*center_point[1] + theta[2])    
    theta[2] += center_point[2] - z_est

    return theta

for idx, row in grids_data.iterrows():
    nearby_data = grids_data.loc[
        (grids_data["x"] >= row["x"] - WINDOW_SIZE) & (grids_data["x"] < row["x"] + WINDOW_SIZE) &
        (grids_data["y"] >= row["y"] - WINDOW_SIZE) & (grids_data["y"] < row["y"] + WINDOW_SIZE)
    ]
    
    if nearby_data.index.size < MIN_POINTS:
        continue

    theta = fit_plane_mmse((row["x"], row["y"], row["level"]), nearby_data[["x", "y", "level"]].values)
    gradient = theta[:2] # Gradient of an adjusted plane is (a, b, -1) -- Here we omit the -1 part.
    grids_data.loc[idx, "arrow"] = Point(gradient)
    grids_data.loc[idx, "arrow_x"] = gradient[0]
    grids_data.loc[idx, "arrow_y"] = gradient[1]
    


In [130]:
# Visualizing the Step

m = folium.Map(location=[47.5515, -122.265], zoom_start=21)

popup = folium.GeoJsonPopup(fields=["name"])
folium.GeoJson(
    grids_df.to_crs(epsg=4326).to_json(),
    name="name",
    popup=popup,
).add_to(m)

transformer = pyproj.Transformer.from_crs("EPSG:3857", "EPSG:4326", always_xy=True)

ARROW_SIZE = 5

for _, row in grids_data.to_crs(epsg=4326).dropna(subset=["arrow"]).iterrows():
    folium.Marker(
        location=[row["coordinate"].y, row["coordinate"].x],
        tooltip=f"Localized: {row["ssid"]} - {row["bssid"]}",
        popup=f'level: {row["level"]}',
        icon=folium.Icon(color="red")
    ).add_to(m)
    if isinstance(row["arrow"], Point):
        arrow_dx, arrow_dy = transformer.transform(row["arrow"].x * ARROW_SIZE, row["arrow"].y * ARROW_SIZE)
        # To simulate the lines, create a line from the dot and add the vector's x,y
        folium.PolyLine(
            locations=[
                [row["coordinate"].y, row["coordinate"].x],
                [row["coordinate"].y + arrow_dy, row["coordinate"].x + arrow_dx]
            ],
            color="#000000",
            weight=3
        ).add_to(m)


m

## Combining the arrows

Once we've determined the direction of the AP from each measurement point, locate the AP at the position to which all arrows point.

The AP is positioned at the location that minimizes the sum-squared angular error from the arrows, weighted by the SNR at the measurement point. In other words, we must find a place in the space where the direction error from the arrow to the possible place of the AP is minimal amongst all planes.

To find this AP position, we can apply one of the following methods:

1. Run an extensively search meter by meter, in a 100m x 100m square around the maximum RSS, to find the place of minimum error.
    1. This makes it an extremely slow performance algorithm  
    2. Possibly results in the most accurate result

2. Run an optimization algorithm that tries to minimize the error function
    1. Tradoff between performance and accuracy
    2. Can end up in a local minimum.


We'll go with the second approach, using an optimization algorithm -- When porting to Ruby, we need to find a library that does what scipy.optimization.minimize does.


### Error Formula

First, we need to calculate the vector from the center point to the estimates position of the AP.

Given two points in a 2D space, the vector from point A to B is: (Bx - Ax; By - Ay).


Then, to calculate the angle between the AP vector, and the computed gradient arrow, we use the following rule:

(Ax, Ay) Dot (Bx, By) = cos(a) * A_magnitude * B_magnitude

Therefore, 

* cos(a) = \[(Ax * Bx) + (Ay * By) / (A_mag * B_mag)]


For the return part, we can use **(1 - cos(a))^2**. When both vectors are perfectly aligned, their angle will be zero, which means cos(a) = 1, so by subtracting by 1, and squaring it, we ensure our target value.


In [145]:
def error_function(ap_position, planes):
    x_ap, y_ap = ap_position
    total_error = 0

    for _, plane in planes.iterrows():
        arrow_dx, arrow_dy = transformer.transform(plane["arrow_x"], plane["arrow_y"])
        arrow_mag = math.sqrt(arrow_dx**2 + arrow_dy**2)
        
        x_plane, y_plane = plane["x"], plane["y"]
    
        x_vector, y_vector = x_ap - x_plane, y_ap - y_plane
        vector_mag = math.sqrt(x_vector**2 + y_vector**2)
        if arrow_mag == 0 or vector_mag == 0:
            continue
        
        total_error += (1 - (x_vector * arrow_dx + y_vector * arrow_dy) / ((vector_mag * arrow_mag))**2)

    return total_error

with_arrow_data = grids_data.dropna(subset="arrow")
initial_position = with_arrow_data.loc[with_arrow_data["level"] == with_arrow_data["level"].max()].iloc[0]

res = scipy.optimize.minimize(
    error_function, 
    x0=initial_position[["x", "y"]].values, 
    args=(with_arrow_data, ),
    method="L-BFGS-B"
)

print(res)
ap_x, ap_y = res.x

  message: ABNORMAL_TERMINATION_IN_LNSRCH
  success: False
   status: 2
      fun: 40.99941507747644
        x: [-1.361e+07  6.033e+06]
      nit: 1
      jac: [ 2.352e+18  1.394e+18]
     nfev: 108
     njev: 36
 hess_inv: <2x2 LbfgsInvHessProduct with dtype=float64>


In [146]:
m = folium.Map(location=[47.5515, -122.265], zoom_start=21)
x, y = transformer.transform(ap_x, ap_y)
folium.Marker(
        location=[y, x],
        tooltip=f"Localized",
        icon=folium.Icon(color="green")
    ).add_to(m)

m